# Interactive exploration of SAP HANA flights data
This app provides a light-weight view of data in a SAP HANA database containing flight data (the SFLIGHT data). The data is stored in a normalized data model, meaning that each table pertains to a single aspect of flight bookings. 

This app allows you to sanity-check the data stored in the tables in the SAP HANA database and explore the data. 

The app is split into the following sections:
1. Select the table you want to explore from the SFLIGHT database.
2. View statistics of the selected table.
3. Select a column of interest in the selected table and view the distribution of values in that column as a pie chart.
4. Select a column of interest containing numeric values and a categorical column to generate a box plot showing the numeric column values as a box plot for each category.

Some business insight questions that can be answered via this app are:
- Which carrier types yield highest ticket prices? (Use the SFLIGHT table and the carrid and price columns.)
- Which carrier types have most seats on e.g. economy class? (Use the SFLIGHT table and the carrid and seatsmax columns.)
- How much cargo can the different cargo types carry? (Use the SCPLANE table and the planetype and cargomax columns.)
- Which travel agency makes most bookings? (Use the SBOOK table and the agencynum column.)
- Where do customers reside? (Use the SCUSTOM table and the country or city column.)

In [ ]:
from functools import partial

import sqlalchemy_hana
from sqlalchemy import create_engine
from sqlalchemy import text

import pandas as pd

import ipywidgets as widgets
import plotly.express as px

import configparser

from hdbcli import dbapi

In [ ]:
config = configparser.ConfigParser()
config.read("credentials/config.txt")
pw = config["sap_hana"]["password"]
user = config["sap_hana"]["username"]
host = config["sap_hana"]["host"]

In [ ]:
from IPython.display import display, HTML

display(HTML("<style>.jp-OutputArea-output {display:flex}</style>"))

In [ ]:
engine = create_engine(
    f"hana://{user}:{pw}@{host}:443",
    echo=False,
    future=True,
)

In [ ]:
try:
    with engine.begin() as conn:
        tables_df = pd.read_sql(text("SELECT TABLE_NAME FROM SYS.M_TABLES where schema_name='SFLIGHT'"), conn)
except DBAPIError as e:
    print(
        "".join(
            [
                "Engine connection did not succeed.",
                " This is likely due to the SAP HANA database not running or your SAP HANA database not allowing incoming traffic from your IP address.",
                " Go to SAP HANA Cloud and find the relevant instance to start it or configure allowed inbound traffic.",
            ]
        )
    )

## Select the table to inspect

In [ ]:
table_selector = widgets.Dropdown(
    options=tables_df["table_name"],
    value=tables_df["table_name"][0],
    disabled=False,
)

table_selector_label = widgets.Label("Table to inspect:")

widgets.HBox([table_selector_label, table_selector])

In [ ]:
def get_selected_table(table_selector, engine, widg_response=None):
    with engine.begin() as conn:
        df_selected_table = pd.read_sql(text(f"select * from sflight.{table_selector.value}"), conn)
    return df_selected_table


def get_selected_table_top(table_selector, engine, top=10):
    with engine.begin() as conn:
        df_selected_table = pd.read_sql(
            text(f"select * from (select top {top} * from sflight.{table_selector.value}) as t"), conn
        )
    return df_selected_table

In [ ]:
def show_selected_table_head(table_selector, engine, output, widg_response=None):
    output.clear_output()
    df = get_selected_table_top(table_selector, engine)
    df_head = df.head()
    with output:
        display(df_head)
        output.df_head = df_head

In [ ]:
class CurrentTable:
    def __init__(self, table_selector, engine):
        self.table_selector = table_selector
        self.engine = engine
        self.df = None
        self.table_name = table_selector.value

    def update_df(self):
        if (self.table_name != self.table_selector.value) or (self.df is None):
            self.df = get_selected_table(self.table_selector, self.engine)
            self.table_name = table_selector.value

In [ ]:
def show_stats_for_selected_table(current_table, output, widg_response=None):
    output.clear_output()
    with output:
        print("Retrieving data")
        current_table.update_df()
        output.clear_output()
        display(current_table.df.describe(include="all"))

In [ ]:
current_table = CurrentTable(table_selector, engine)

In [ ]:
df_head_output = widgets.Output()
df_stats_output = widgets.Output()

In [ ]:
table_selector.observe(partial(show_selected_table_head, table_selector, engine, df_head_output), names="value")
table_selector.observe(partial(lambda output, widg: output.clear_output(), df_stats_output), names="value")

### View first rows of table

In [ ]:
show_selected_table_head(table_selector, engine, df_head_output)

In [ ]:
display(df_head_output)

## Retrieve data and view descriptive statistics
Click the below button to retrieve all the available data if you are happy with the selected table based on the first few rows. This will also show statistics for the selected table.

Note that, for large tables, this can take some time.

In [ ]:
layout = widgets.Layout(width="auto", height="40px")
retrieve_data_button = widgets.Button(description="Retrieve all data and view statistics", layout=layout)
retrieve_data_button.on_click(partial(show_stats_for_selected_table, current_table, df_stats_output))
display(retrieve_data_button)

In [ ]:
display(df_stats_output)

## Visualize data

### Pie chart
Select a column for which to view the distribution of values. You can explore the different columns in the selected table.

In [ ]:
class ColumnSelector:
    def __init__(self, df_head_output_field, label_text="Column to visualize:"):
        self.df_head_output_field = df_head_output_field
        self.label_text = label_text
        self.selector = widgets.Dropdown(
            options=self.df_head_output_field.df_head.columns,
            value=self.df_head_output_field.df_head.columns[0],
            disabled=False,
        )

    def update_column_selector(self):
        self.selector = widgets.Dropdown(
            options=self.df_head_output_field.df_head.columns,
            value=self.df_head_output_field.df_head.columns[0],
            disabled=False,
        )

    def display_in_output(self, output, widget_obj=None):
        self.update_column_selector()
        with output:
            output.clear_output()
            column_selector_label = widgets.Label(self.label_text)
            display(widgets.HBox([column_selector_label, self.selector]))

In [ ]:
column_selector_output = widgets.Output()
column_selector = ColumnSelector(df_head_output)
column_selector.display_in_output(column_selector_output)
table_selector.observe(partial(column_selector.display_in_output, column_selector_output), names="value")

display(column_selector_output)

In [ ]:
pie_output = widgets.Output()


def show_pie_chart(current_table, column_selector, output, widg_obj=None):
    with output:
        output.clear_output()
        current_table.update_df()
        value_counts = current_table.df[column_selector.selector.value].value_counts()

        fig = px.pie(
            names=value_counts.index,
            values=value_counts.to_numpy(),
            title=f"Counts of values in {column_selector.selector.value}",
        )
        fig.show()

In [ ]:
table_selector.observe(partial(lambda output, widg: output.clear_output(), pie_output), names="value")
pie_chart_button = widgets.Button(description="Generate pie chart", layout=layout)
pie_chart_button.on_click(partial(show_pie_chart, current_table, column_selector, pie_output))
display(pie_chart_button)

In [ ]:
display(pie_output)

## Box plot
To understand historical relationships between a numeric value of interest and it's distribution for different categories, select the relevant columns below and generate a box plot.

In [ ]:
column_selector_x_output = widgets.Output()
column_selector_x = ColumnSelector(df_head_output, label_text="Category to group by:")
column_selector_x.display_in_output(column_selector_x_output)
table_selector.observe(partial(column_selector_x.display_in_output, column_selector_x_output), names="value")

display(column_selector_x_output)

In [ ]:
column_selector_y_output = widgets.Output()
column_selector_y = ColumnSelector(df_head_output, label_text="Numeric variable of interest:")
column_selector_y.display_in_output(column_selector_y_output)
table_selector.observe(partial(column_selector_y.display_in_output, column_selector_y_output), names="value")

display(column_selector_y_output)

In [ ]:
boxplot_output = widgets.Output()


def show_boxplot(current_table, column_selector_x, column_selector_y, output, widg_obj=None):
    with output:
        output.clear_output()
        current_table.update_df()

        fig = px.box(current_table.df, x=column_selector_x.selector.value, y=column_selector_y.selector.value)
        fig.show()

In [ ]:
table_selector.observe(partial(lambda output, widg: output.clear_output(), pie_output), names="value")
boxplot_button = widgets.Button(description="Generate boxplot", layout=layout)
boxplot_button.on_click(partial(show_boxplot, current_table, column_selector_x, column_selector_y, boxplot_output))
display(boxplot_button)

In [ ]:
display(boxplot_output)